### installing beautifulsoap and lxml for html scraping, and importing relevent libraries

In [1]:
pip install beautifulsoup4

     |████████████████████████████████| 122kB 21.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install lxml


     |████████████████████████████████| 5.5MB 4.9MB/s eta 0:00:01     |████████▏                       | 1.4MB 4.9MB/s eta 0:00:01     |████████████████████████▏       | 4.2MB 4.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
import requests
from random import randint
from bs4 import BeautifulSoup
print('libs loaded')

libs loaded


### Getting the data on toronto neighborhood postal codes from wikipedia and creating a pandas dataframe

In [2]:
wikiurl="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [3]:
soup = BeautifulSoup(response.text, 'html.parser')
indiatable=soup.find('table',{'class':"wikitable"})

In [5]:
df=pd.read_html(str(indiatable))

df=pd.DataFrame(df[0])
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


### cleaning the data geeing rid of all the empty rows

In [6]:
###copying the original dataframe

df2 = df

In [7]:
### dropping empty cells and reseting the index


df2 = df2.drop(df2[df2.Borough == 'Not assigned'].index)

In [8]:
df2 = df2.reset_index(drop=True)

In [9]:
df2

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [10]:
df2.shape

(103, 3)

### adding coordinates to the dataframe
### using the csv file online

In [11]:
csvurl='https://cocl.us/Geospatial_data'

In [12]:
coordata = pd.read_csv(csvurl)

In [13]:
coordata

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


### merging the dataframes by the postal code

In [14]:
df4 = pd.merge(df2, coordata, on='Postal Code')

In [16]:
df4

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


### counting the number of different postel codes in different boroughs

In [15]:
df_b = df4.Borough.value_counts()

df_b

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

In [59]:
### creating color dataframe for each neighbourhood to use with folium maps

colors = ["red", "blue", "green", "gray", "orange", "cadetblue", "lightred", "purple", "black", "darkgreen", "lightblue", "darkpurple", "white", "darkred", "darkblue", "lightgreen", "biege", "pink", "lightgray"]

dfcolor=df4.Borough.unique()
colfornei=colors[0:len(dfcolor)]


In [60]:
nei=pd.DataFrame(dfcolor, columns = ['Borough'])
col=pd.DataFrame(colfornei, columns = ['Color'])
neicol=pd.concat([nei, col], axis=1)
neicol=neicol.set_index(['Borough'])


In [61]:

neicol

,Color
Borough,
North York,red
Downtown Toronto,blue
Etobicoke,green
Scarborough,gray
East York,orange
York,cadetblue
East Toronto,lightred
West Toronto,purple
Central Toronto,black


### Creating a map of all neighbourhoods in toronto

In [20]:
pip install folium

     |████████████████████████████████| 94 kB 5.2 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [62]:
import folium

In [63]:
### setting the initial coordinates of the map with the first latitude and longitude

latitude = df4.iloc[1,3]
longitude = df4.iloc[1,4]

In [64]:
### create map of toronto boroughs using latitude and longitude values, coloring each borough
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

### adding markers to map
for lat, lng, Borough, Neighbourhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color=neicol._get_value(Borough,'Color'),
        key_on = Borough,
        fill=True,
        fill_color=neicol._get_value(Borough,'Color'),
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### creating a map of specific boroughs

### Creating a map of borrows containing the word "Toronto" in them

In [65]:
df_ntor = df4[df4['Borough'].str.contains("Toronto")]
df_ntor = df_ntor.reset_index(drop=True)

In [66]:
df_ntor

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [67]:
### counting the number of different postel codes in "toronto" containing borrows
df_bt = df_ntor.Borough.value_counts()
df_bt

Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

In [68]:
### setting the initial coordinates of the map with the first latitude and longitude

latitudetb = df_ntor.iloc[1,3]
longitudetb = df_ntor.iloc[1,4]

In [70]:
### create map of Toronto boroughs containing the word "toronto" in them using latitude and longitude values
map_torontob = folium.Map(location=[latitudetb, longitudetb], zoom_start=11)

### add markers to map - coloring each borrow in a different color
for lat, lng, Borough, Neighbourhood, pcode in zip(df_ntor['Latitude'], df_ntor['Longitude'], df_ntor['Borough'], df_ntor['Neighbourhood'], df_ntor['Postal Code']):
    label = '{}, {}, {}'.format(Neighbourhood, Borough, pcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color=neicol._get_value(Borough,'Color'),
        fill=True,
        fill_color=neicol._get_value(Borough,'Color'),
        fill_opacity=0.5,
        parse_html=False).add_to(map_torontob)  
    
map_torontob

### using Foursqure to explore nearby businesses in one nighborhood from "toronto" boroughs dataframe

In [71]:
### loading relevent libraries
import json ### to handle the results from foursquare in json file
from pandas.io.json import json_normalize ### tranform json file into a pandas dataframe
import requests

In [72]:
### defining Foursqure credentials

client_id = 'FWLLSZJCW1CQD3EKW0L3P0B15QSR1NID4JX2EKZRPXS0JPW5' 
client_secret = 'VRRBWFUJ0NRCA555QXSNNKWGHC0M55OAUQRMRQSMR5QXUCZZ' 
version = '20180605' 
limit  = 100 


In [73]:
### getting details of the first neighbourhood in the dataframe

In [74]:
neighborhood_latitude = df_ntor.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_ntor.loc[0, 'Longitude'] # neighborhood longitude value

Neighbourhood_name = df_ntor.loc[0, 'Neighbourhood'] # Neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(Neighbourhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [75]:
### getting the data from foursquare api, limit results to 100 - radius of 500 meters from coordinates


limit = 100 



radius = 500 

### creating the url for the request


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    client_id, 
    client_secret, 
    version, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    limit)
url 


'https://api.foursquare.com/v2/venues/explore?&client_id=FWLLSZJCW1CQD3EKW0L3P0B15QSR1NID4JX2EKZRPXS0JPW5&client_secret=VRRBWFUJ0NRCA555QXSNNKWGHC0M55OAUQRMRQSMR5QXUCZZ&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [76]:
### Getting the data from foursquare in a JSON file

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600d9486dd4ff939fa3172a0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 45,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [77]:
# function to extract the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [78]:
### Getting only relevant data from JSON file
venues = results['response']['groups'][0]['items']


### Transforming the JSON file to a dataframe 
nearby_venues = json_normalize(venues)

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.columns =['Busniess Name', 'Category', 'Latitude', 'Longitude']

nearby_venues.head(50)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,Busniess Name,Category,Latitude,Longitude
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980
5,Corktown Common,Park,43.655618,-79.356211
6,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
7,The Extension Room,Gym / Fitness Center,43.653313,-79.359725
8,The Distillery Historic District,Historic Site,43.650244,-79.359323
9,SOMA chocolatemaker,Chocolate Shop,43.650622,-79.358127


In [85]:
nearby_venues.shape

(45, 4)

In [ ]:
### getting information on all nearby venues for all the neighbourhoods on "toronto" boroughs

In [86]:
### a function that makes a call to foursquare api and gets a list of nearby venues for a given location


def getNearbyVenues(names, latitudes, longitudes, radius=250):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        ### creating a url for requests
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(client_id, client_secret, version, lat, lng, radius, limit)
            
        ### make the GET request for results
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        ### return information for each nearby venue
        venues_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [87]:
### getting nearby venues for all the toronto containing boroughs

torontob_venues = getNearbyVenues(names=df_ntor['Neighbourhood'],
                                   latitudes=df_ntor['Latitude'],
                                   longitudes=df_ntor['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [88]:
print(torontob_venues.shape)
torontob_venues.head(50)

(624, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
3,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
5,"Regent Park, Harbourfront",43.654260,-79.360636,The Extension Room,43.653313,-79.359725,Gym / Fitness Center
6,"Regent Park, Harbourfront",43.654260,-79.360636,Enoch Turner Schoolhouse,43.652873,-79.361672,History Museum
7,"Regent Park, Harbourfront",43.654260,-79.360636,Buster's Sea Cove Truck,43.653647,-79.357878,Food Truck
8,"Regent Park, Harbourfront",43.654260,-79.360636,studio b,43.653966,-79.361662,Furniture / Home Store
9,"Regent Park, Harbourfront",43.654260,-79.360636,The Corktown Kitchen,43.653316,-79.362140,Gastropub


In [89]:
### Listing the number of venues for each location

busnums = torontob_venues.groupby('Neighbourhood').count()
busnums.drop(['Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue Latitude', 'Venue Longitude', 'Venue Category'], axis='columns', inplace=True)
busnums.sort_values(['Venue', 'Neighbourhood'], inplace=True, ascending=(False, True))



In [87]:
busnums.columns =['Number Of Venues']
busnums

,Number Of Venues
Neighbourhood,
"Commerce Court, Victoria Hotel",62
"Toronto Dominion Centre, Design Exchange",59
"First Canadian Place, Underground city",46
"Kensington Market, Chinatown, Grange Park",39
"Garden District, Ryerson",38
"Harbourfront East, Union Station, Toronto Islands",38
"Richmond, Adelaide, King",36
Church and Wellesley,35
Stn A PO Boxes,35


### Counting how many kinds of businesses there are in "toronto" boroughs

In [107]:
print('There are {} unique kinds of business in the "Toronto" containing boroughs.'.format(len(torontob_venues['Venue Category'].unique())))

There are 158 unique business in the "Toronto" containing boroughs.


## Counting how many businesses in total there are in "toronto" boroughs

In [164]:
print('There are {} total business in the "Toronto" containing boroughs.'.format(len(torontob_venues)))

There are 624 total business in the "Toronto" containing boroughs.


### Creating a top 5 list of most common businesses in the area

In [99]:
topcats = torontob_venues.groupby('Venue Category').count()
topcats.drop(['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue Latitude', 'Venue Longitude'], axis='columns', inplace=True)

In [100]:
topcats.sort_values(['Venue', 'Venue Category'], inplace=True, ascending=(False, True))
topcats.columns =['Number Of Venues']

topcats.head(5)

,Number Of Venues
Venue Category,
Coffee Shop,69
Café,32
Restaurant,19
Bakery,14
Bar,14


### creating a north york only map

In [118]:
dt_ny = df4.loc[df4['Borough'] == 'North York']

In [119]:
### setting the initial coordinates of the map with the first latitude and longitude

latitudeny = dt_ny.iloc[1,3]
longitudeny = dt_ny.iloc[1,4]

In [123]:
# create map of North York only using latitude and longitude values
map_nytoronto = folium.Map(location=[latitudeny, longitudeny], zoom_start=12)

# add markers to map
for lat, lng, Borough, Neighbourhood in zip(dt_ny['Latitude'], dt_ny['Longitude'], dt_ny['Borough'], dt_ny['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, Borough)
    label = folium.Popup(label, parse_html=False)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color=neicol._get_value(Borough,'Color'),
        fill=True,
        fill_color=neicol._get_value(Borough,'Color'),
        fill_opacity=0.7,
        parse_html=False).add_to(map_nytoronto)  
    
map_nytoronto

In [124]:
### getting information on nearby business from foursquare for North York


nyork_venues = getNearbyVenues(names=dt_ny['Neighbourhood'],
                                   latitudes=dt_ny['Latitude'],
                                   longitudes=dt_ny['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [165]:
print('There are {} unique kinds of business in North York borough.'.format(len(nyork_venues['Venue Category'].unique())))

There are 64 unique kinds of business in North York borough.


In [129]:
print('there are {} business in North York borough'.format(len(nyork_venues)))

there are 109 business in North York borough


In [142]:
### Listing the number of venues for each location

nybusnums = nyork_venues.groupby('Neighbourhood').count()
nybusnums.drop(['Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue Latitude', 'Venue Longitude', 'Venue Category'], axis='columns', inplace=True)
nybusnums.sort_values(['Venue', 'Neighbourhood'], inplace=True, ascending=(False, True))



In [166]:
print('top 5 business locations in North York borough are:')
nybusnums.head(5)

top 5 business locations in North York borough are:


,Venue
Neighbourhood,
"Fairview, Henry Farm, Oriole",42
Don Mills,15
"Bedford Park, Lawrence Manor East",14
"Bathurst Manor, Wilson Heights, Downsview North",10
"Lawrence Manor, Lawrence Heights",6


In [162]:
### making a list of top 3 most common businesses in north york borough

nytopcats = nyork_venues.groupby('Venue Category').count()
nytopcats.drop(['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue Latitude', 'Venue Longitude'], axis='columns', inplace=True)
nytopcats.columns =['Number Of Venues in North York']



In [167]:
nytopcats.sort_values(['Number Of Venues in North York', 'Venue Category'], inplace=True, ascending=(False, True))



In [168]:
print('top 3 business categories in North York are:')
nytopcats.head(3)

top 3 business categories in North York are:


,Number Of Venues in North York
Venue Category,
Clothing Store,10
Coffee Shop,9
Restaurant,6
